# **Projeto - Computação em Nuvem**

### Eiki Luis Yamashiro, 6º semestre, Engenharia de Computação

### Instalação do boto3

No terminal:

``` $ pip install boto3 ```

Documentação: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In [1]:
import sys
import boto3
import time
OHIO_NAME = "ohio_key_pair"
NORTH_VIRGINIA_NAME = "north_virginia_key_pair"

### Instalação do aws cli

#### Windows 10

No terminal, copie e cole o seguinte comando:

``` $ msiexec.exe /i https://awscli.amazonaws.com/AWSCLIV2.msi ```

![](awscli_install.png)

Após a instalação, verificar a versão:

``` $ aws --version ```

``` aws-cli/2.2.45 Python/3.8.8 Windows/10 exe/AMD64 prompt/off ```

Documentação: https://aws.amazon.com/pt/cli/

### Amazon Web Services Command Line Interface

A AWS Command Line Interface (AWS CLI) é uma ferramenta de código aberto que permite que você interaja com os serviços da AWS usando comandos em seu shell de linha de comando. Com configuração mínima, o AWS CLI permite que você comece a executar comandos que implementam funcionalidade equivalente àquela fornecida pelo AWS Management Console baseado em navegador a partir do prompt de comando em seu programa de terminal.

Documentação: https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html

### Política de Ohio

Para criar a instância em Ohio foi necessário alterar as políticas associadas ao usuário do aws no IAM. Criou-se uma nova política AdminOhio com as seguintes configurações:

```python
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "*",
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "aws:RequestedRegion": "us-east-2"
                }
            }
        }
    ]
}
```

### Amazon EC2

O Amazon Simple Storage Service (Amazon S3) é um serviço de armazenamento de objetos que oferece escalabilidade, disponibilidade de dados, segurança e desempenho.

Documentação: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-examples.html

**Region:**
- North Virginia: us-east-1
- Ohio: us-east-2

**Hardware:**
- t2.micro

**AMI North Virginia (us-east-1):**
- Ubuntu Server 20.04 LTS (HVM), SSD Volume: ami-09e67e426f25ce0d7 (x86)
- Ubuntu Server 16.04 LTS (HVM), SSD Volume: ami-0ee02acd56a52998e (x86)

**AMI Ohio (us-east-2):**
- Ubuntu Server 20.04 LTS (HVM), SSD Volume: ami-00399ec92321828f5 (x86)
- Ubuntu Server 18.04 LTS (HVM), SSD Volume: ami-0b9064170e32bde34 (x86)

Estabelecer uma conexão com aws ec2 e boto3:

Descrever instâncias:

In [31]:
#ec2.describe_instances()

In [28]:
def create_client(region, tipo="ec2"):
    
    return boto3.client(tipo, region_name=region)
    
def create_key_pair(client, name):
    
    name_flag = True
    
    for KeyPair in client.describe_key_pairs()["KeyPairs"]:

        if KeyPair["KeyName"] == name:
            
            print("KeyName ", name, " already exists")
            name_flag = False
            return False
            break
            
    if name_flag:
        
        response = client.create_key_pair(KeyName=name)
        print("KeyPair ",name," created successfully")
        return response

def delete_key_pair(client , name):
    
    for KeyPair in client.describe_key_pairs()["KeyPairs"]:
        
        if KeyPair["KeyName"] == name:

            response = client.delete_key_pair(KeyName=name)
            print("KeyPair ",name," deleted successfully")
            
            return True
            
            break

    print("Unable to delete KeyPair ", name)
    return False

def create_instance(hw, os, region, client, KeyPair, script, security_group_id):
    
    instance = client.run_instances(
        ImageId= os, 
        MinCount=1,
        MaxCount=1,
        InstanceType= hw,
        KeyName = KeyPair["KeyName"],
        SecurityGroupIds = [security_group_id],
        TagSpecifications=[
        {
          "ResourceType": "instance",
          "Tags": [
            {
              "Key": "Name",
              "Value": f'{region}-instance'
            }
          ]
        }
      ],
        UserData = script
    )
    
    waiter = client.get_waiter('instance_status_ok')
    
    waiter.wait(InstanceIds=[instance["Instances"][0]["InstanceId"]])
    
    print(f'Instance {instance["Instances"][0]["InstanceId"]} created successfully')
    
    return instance

def get_ip(instance, instance_id):
    
    for e in instance.cursor.client.describe_instances()["Reservations"]:
        if e["Instances"][0]["InstanceId"] == instance_id:
            saida = e["Instances"][0]["PublicIpAddress"]
            
    return saida

def launch_config(client, name, image, security_group, cursor):

    return client.create_launch_configuration(LaunchConfigurationName = name,
                                      ImageId = image.id,
                                      SecurityGroups = [security_group],
                                      InstanceType = 't2.micro',
                                      KeyName = cursor.keypair["KeyName"])

def auto_scalling_group(ec2, auto_scalling, target_groupNV):
    
    z = []
    for zones in ec2.describe_availability_zones()['AvailabilityZones']:
        z.append(zones["ZoneName"])
    
    return auto_scalling.create_auto_scaling_group(
        AutoScalingGroupName= "as_name",
        LaunchConfigurationName= "launch_config",
        MinSize=1,
        MaxSize = 3,
        TargetGroupARNs = [target_groupNV.arn],
        AvailabilityZones = z
    )

def delete_launch_config(client, name):
    return client.delete_launch_configuration(LaunchConfigurationName = name)

def delete_as_group(client, name):
    
    return client.delete_auto_scaling_group(AutoScalingGroupName=name,
                                    ForceDelete = True)

def create_auto_scaling_policy(client, auto_scaling_group_name, load_balancer, target_group):
    return client.put_scaling_policy(
        AutoScalingGroupName=auto_scaling_group_name,
        PolicyName='TargetTrackingScaling',
        PolicyType='TargetTrackingScaling',
        TargetTrackingConfiguration={
            'PredefinedMetricSpecification': {
                'PredefinedMetricType': 'ALBRequestCountPerTarget',
                'ResourceLabel': f'{load_balancer.arn[load_balancer.arn.find("app"):]}/{target_group.arn[target_group.arn.find("targetgroup"):]}'
            },
            'TargetValue': 30
        }
    )

In [3]:
class Cursor():
    
    def __init__(self , client, region, name, keypair=None):
        self.client = client
        self.region = region
        self.name = name
        self.keypair = keypair
        
    def createKeyPair(self):
        self.keypair = create_key_pair(self.client, self.name)
        
    def setKeyPair(self, keypair):
        self.keypair = keypair
        
    def deleteKeyPair(self):
        response = input(f"Are you sure you want to delete {self.name}? (y/n)")
        if response == 'y' or response == "yes":
            
            delete_key_pair(self.client, self.name)
            
        else: 
            print("Operation canceled")
            
    def get_vpc_id(self):
        return self.client.describe_vpcs()['Vpcs'][0]['VpcId']

In [4]:
class Instance():
    
    def __init__(self, cursor, region):
        
        self.cursor = cursor
        self.counter = 0
        self.region = region
        
        if region == "ohio":
            self.hw = "t2.micro"
            self.os = "ami-00399ec92321828f5"
            
        if region == "north-virginia":
            self.hw = "t2.micro"
            self.os = "ami-09e67e426f25ce0d7"
        
    def setHW(self, hw):
        self.hw = hw
        
    def setOS(self, os):
        self.os = os
        
    def create_instance(self, script):
        
        self.script = script
        
        if self.hw == None:
            print("Set the instance hardware!")
            
        if self.os == None:
            print("Set the instance operational system!")
        
        self.instance = create_instance(self.hw, 
                                        self.os, 
                                        self.cursor.region, 
                                        self.cursor.client, 
                                        self.cursor.keypair,
                                        self.script,
                                        self.security_group['GroupId'],
                                        )
        
        self.id = self.instance["Instances"][0]['InstanceId']
        
    def delete_instance(self, force_delete = False):
        
        response = input(f"Are you sure you want to delete the instance {self.id} from {self.cursor.region}? (y/n)")
        
        if response == 'y' or response == "yes" or force_delete:
            
            response = self.cursor.client.terminate_instances(InstanceIds=[self.id])
            
            waiter = self.cursor.client.get_waiter('instance_terminated')
    
            waiter.wait(InstanceIds=[self.id])
            
            print("Instance ",self.id," deleted successfully")
            
        else: 
            print("Operation canceled")
        
    def create_security_group(self, permissions_config):
       
        self.vpc_id = self.cursor.client.describe_vpcs()["Vpcs"][0]["VpcId"]
        
        self.security_group_name = f'security-group-{str(time.time())[-3:]}'
        
        self.security_group = self.cursor.client.create_security_group(GroupName = self.security_group_name,
                                                                       Description = 'Security Group of instance')
        
        self.cursor.client.authorize_security_group_ingress(GroupId = self.security_group['GroupId'],
                                                            IpPermissions = permissions_config)
        print("Security Group ",self.security_group_name," created successfully")
        
        self.counter += 1
        
    def delete_security_group(self):
        response = input(f"Are you sure you want to delete the security group {self.security_group['GroupId']} from {self.cursor.region}? (y/n)")
        
        if response == 'y' or response == "yes":
            for e in cursorOH.client.describe_security_groups()["SecurityGroups"]:
            
                if e["GroupName"] == self.security_group_name:
                    
                    r = self.cursor.client.delete_security_group(GroupName=e["GroupName"], GroupId=e["GroupId"])
                    print("Security Group ",self.security_group['GroupId']," deleted successfully")    

In [5]:
# Imagem

def create_image(instance):
    
    response = instance.cursor.client.create_image(InstanceId=instance.id, Name=f'{instance.cursor.region}-image')
    return response["ImageId"]
    
def delete_image_function(client, image_id):

    for image in client.describe_images():
        
        if image["ImageId"] == image_id:
            client.deregister_image(ImageId = image_id)
            print("Image ", image_id, "deleted successfuly")
            return True
    
    return False
    
class Image():
    
    def __init__(self, instance):
        self.id = create_image(instance)
        self.cursor = instance.cursor
        self.instance = instance
        
    def delete_image(self):
        
        delete_image_function(self.cursor.client, self.id)

In [26]:
# Load Balancer

class LoadBalancer():

    # cliente do load balancer e security group
    def __init__(self, client, instance):
        self.name = f"load-balancer-{instance.security_group_name}"
        self.client = client
        self.security_group = instance.security_group
        
    def create(self, client):
        
        l = []
        for subnet in client.describe_subnets()["Subnets"]:
            l.append(subnet['SubnetId'])
            
        print("Creating Load Balancer")
        waiter = self.client.get_waiter('load_balancer_available')
        response = self.client.create_load_balancer(Name = self.name,
                                                    Type='application',
                                                    Subnets = l,
                                                    Scheme='internet-facing',
                                                    SecurityGroups = [self.security_group["GroupId"]],
                                                    IpAddressType = "ipv4")
        
        print("Load Balancer created successfuly")
        print(f"DNSName {response['LoadBalancers'][0]['DNSName']}")
        print(f"LoadBalancerArn {response['LoadBalancers'][0]['LoadBalancerArn']}")
        
        self.arn = response['LoadBalancers'][0]['LoadBalancerArn']
        
    def delete(self):
        self.client.delete_load_balancer(LoadBalancerArn=self.arn)
        
        # Criar waite do delete: waiter.wait(LoadBalancerArns=self.arn)
        print(f"Load Balancer {self.name} deleted successfuly")            

In [25]:
# Target Group
    
class TargetGroup():
    
    # Client do Load Balancer!
    def __init__(self, client_load_balancer, name, protocol, port, target_type, vpc_id):
        self.client = client_load_balancer
        self.name = name
        self.protocol = protocol
        self.port = port
        self.target_type = target_type
        self.vpc_id = vpc_id
        
        response = self.client.create_target_group(Name  = self.name,
                                                   Protocol  = self.protocol,
                                                   Port  = self.port,
                                                   TargetType = self.target_type,
                                                   VpcId  = self.vpc_id,
                                                   HealthCheckPath = '/admin/',
                                                   Matcher = {
                                                                "HttpCode": "200,302"
                                                            })
        
        self.arn = response['TargetGroups'][0]['TargetGroupArn']
        
        print(f"Target Group {response} created successfuly")
        
    def delete_target_group(self):
        self.client.delete_target_grouop(TargetGroupArn = self.arn)
        print(f'Target Group {response} deleted successfuly')

In [8]:
class Listener():
    
    def __init__(self, client, load_balancer_arn, target_group_arn):
            self.load_balancer_arn = load_balancer_arn
            self.target_group_arn = target_group_arn
            self.client = client
            self.summary = self.client.create_listener(LoadBalancerArn = self.load_balancer_arn,
                                                      Protocol = 'HTTP',
                                                      Port = 80,
                                                      DefaultActions=[
                                                        {
                                                            'Type': 'forward',
                                                            'TargetGroupArn': self.target_group_arn

                                                        }
                                                      ])
            
            self.arn = self.summary['Listeners'][0]['ListenerArn']

    def delete(self):
        
        self.client.delete_listener(ListenerArn = self.arn)

In [ ]:
# Create ec2 client conection 
ec2_ohio = create_client("us-east-2")
ec2_north_virginia = create_client("us-east-1")

# Postgres
SECURITY_GROUP_POSTGRES = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 5432,
        'ToPort': 5432,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    }
]

# Postgres user data
USER_DATA_POSTGRES = """
#cloud-config
runcmd:
- cd /
- sudo apt update -y
- sudo apt install postgresql postgresql-contrib -y
- sudo su - postgres
- sudo -u postgres psql -c "CREATE USER cloud WITH PASSWORD 'cloud';"
- sudo -u postgres psql -c "CREATE DATABASE tasks;"
- sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tasks TO cloud;"
- sudo echo "listen_addresses = '*'" >> /etc/postgresql/12/main/postgresql.conf
- sudo echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/12/main/pg_hba.conf
- sudo ufw allow 5432/tcp -y
- sudo systemctl restart postgresql
"""

# -------------------------------------------------------------------------------------------

cursorOH = Cursor(ec2_ohio, "us-east-2","ohio-key-pair")
cursorOH.createKeyPair()

postgresOH = Instance(cursorOH, "ohio")
postgresOH.create_security_group(SECURITY_GROUP_POSTGRES)
postgresOH.create_instance(USER_DATA_POSTGRES)

print(f"Ohio Postgres Instance IP: {get_ip(postgresOH, postgresOH.id)}")

file = open(f'C:/Users/eikis/.ssh/{postgresOH.cursor.keypair["KeyName"]}.pem', "w")
file.write(postgresOH.cursor.keypair["KeyMaterial"])
file.close()

# -------------------------------------------------------------------------------------------

# Django
SECURITY_GROUP_DJANGO = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8080,
        'ToPort': 8080,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    }
]

# Django user data
USER_DATA_DJANGO = f"""
#cloud-config
runcmd:
- sudo apt update
- cd /home/ubuntu 
- git clone https://github.com/EikiYamashiro/tasks.git
- cd tasks
- sed -i "s/node1/{get_ip(postgresOH, postgresOH.id)}/g" ./portfolio/settings.py
- ./install.sh
- sudo ufw allow 8080/tcp -y
- sudo reboot
"""

cursorNV = Cursor(ec2_north_virginia, "us-east-1","north-virginia-key-pair")
cursorNV.createKeyPair()

djangoNV = Instance(cursorNV, "north-virginia")
djangoNV.create_security_group(SECURITY_GROUP_DJANGO)
djangoNV.create_instance(USER_DATA_DJANGO)

print(f"North Virginia Django Instance IP: {get_ip(djangoNV, djangoNV.id)}")

file = open(f'C:/Users/eikis/.ssh/{djangoNV.cursor.keypair["KeyName"]}.pem', "w")
file.write(djangoNV.cursor.keypair["KeyMaterial"])
file.close()

client_load_balancer = create_client("us-east-1", "elbv2")
client_auto_scalling = create_client("us-east-1", "autoscaling")

imageNV = Image(djangoNV)

print(f"Image {imageNV.id} created")

djangoNV.delete_instance(force_delete = True)
print(f"Instance {djangoNV.id} (Django) from {djangoNV.region} deleted")

target_groupNV = TargetGroup(client_load_balancer, "target-group-NV", 'HTTP', 8080, 'instance', cursorNV.get_vpc_id())
print(f"Target Group {target_groupNV.name} created")

load_balancerNV = LoadBalancer(client_load_balancer, djangoNV)
load_balancerNV.create(ec2_north_virginia)
print(f"Load Balancer {load_balancerNV.name} created")

launch_config(client_auto_scalling, "launch_config", imageNV, djangoNV.security_group["GroupId"], cursorNV)
print("Launch Configuration Created")

as_group = auto_scalling_group(ec2_north_virginia, client_auto_scalling, target_groupNV)
print("Auto Scaling Group created")

attach = client_auto_scalling.attach_load_balancer_target_groups(
        AutoScalingGroupName="as_name",
        TargetGroupARNs=[
            target_groupNV.arn,
        ]
    )
print(f"Load Balancer {load_balancerNV.name} attached to Target Group {target_groupNV.name}")

listener = Listener(client_load_balancer, load_balancerNV.arn, target_groupNV.arn)
print("Listener created")

auto_scaling_policy = create_auto_scaling_policy(client_auto_scalling, "as_name", load_balancerNV, target_groupNV)
print("Auto Scaling Poolicy created")

flag = True
while flag:
    print(" ")
    print("--------------------------MANAGER-MENU--------------------------")
    print("Select a action:")
    response = input("(1) Stop and Delete\n(2) Save data")
    
    if response == "1":
        cursorOH.deleteKeyPair()
        postgresOH.delete_instance()
        postgresOH.delete_security_group()

        cursorNV.deleteKeyPair()
        djangoNV.delete_security_group()

        imageNV.cursor.client.deregister_image(ImageId = imageNV.id)
        print(f"Image {imageNV.id} deleted")

        load_balancerNV.delete()

        time.sleep(30)

        target_groupNV.client.delete_target_group(TargetGroupArn = target_groupNV.arn)
        print(f"Target Group {target_groupNV.name} deleted")

        time.sleep(10)

        delete_as_group(client_auto_scalling, "as_name")
        print("Auto Scaling Group deleted")

        time.sleep(10)

        delete_launch_config(client_auto_scalling, "launch_config")
        print("Launch Config deleted")
        
        delete_flag = True
        while delete_flag:
            print("--------------------------MANAGER-MENU--------------------------")
            print("Select a action:")
            response = input("(1) Deploy\n(2) Exit")
            if response == "1":
                delete_flag = False
            elif response == "2":
                delete_flag = False
                flag = False
            
        
    elif response == "2":
        print("Data saved!")
        
    else:
        pass

KeyPair  ohio-key-pair  created successfully
Security Group  security-group-512  created successfully
Instance i-0baad5cee795d27c9 created successfully
Ohio Postgres Instance IP: 3.16.40.3
KeyPair  north-virginia-key-pair  created successfully
Security Group  security-group-615  created successfully
Instance i-078fbfc804d8487e4 created successfully
North Virginia Django Instance IP: 3.93.17.159
Image ami-06ee028f9a48f9841 created
Are you sure you want to delete the instance i-078fbfc804d8487e4 from us-east-1? (y/n)y
Instance  i-078fbfc804d8487e4  deleted successfully
Instance i-078fbfc804d8487e4 (Django) from north-virginia deleted
Target Group {'TargetGroups': [{'TargetGroupArn': 'arn:aws:elasticloadbalancing:us-east-1:202757356229:targetgroup/target-group-NV/c49c7a846f30964e', 'TargetGroupName': 'target-group-NV', 'Protocol': 'HTTP', 'Port': 8080, 'VpcId': 'vpc-1ce22266', 'HealthCheckProtocol': 'HTTP', 'HealthCheckPort': 'traffic-port', 'HealthCheckEnabled': True, 'HealthCheckInterva